## Harmony Py Library
### Job Results STAC data

In [ ]:
import sys
sys.path.append('..')

!{sys.executable} -m pip install -q -r ../requirements/examples.txt

# Install harmony-py requirements.  Not necessary if you ran `pip install harmony-py` in your kernel  
!{sys.executable} -m pip install -q -r ../requirements/core.txt

from harmony import BBox, Client, Collection, Request, Environment

First, let's get a job processing in Harmony.

In [ ]:
harmony_client = Client(env=Environment.UAT)  # assumes .netrc usage

collection = Collection(id='C1234088182-EEDTEST')
request = Request(
    collection=collection,
    spatial=BBox(-165, 52, -140, 77),
    max_results=5
)

job_id = harmony_client.submit(request)
job_id

Harmony-py can return the STAC Catalog URL for a completed job.

In [ ]:
stac_catalog_url = harmony_client.stac_catalog_url(job_id, show_progress=True)

Following the directions for PySTAC (https://pystac.readthedocs.io/en/latest/quickstart.html), we can hook our harmony-py client into STAC_IO.

In [ ]:
from urllib.parse import urlparse
import requests
from pystac import stac_io


def requests_read_method(uri):
    parsed = urlparse(uri)
    if parsed.hostname.startswith('harmony.') or parsed.hostname.startswith('localhost'):
        return harmony_client.read_text(uri)
    else:
        return stac_io.default_read_text_method(uri)

stac_io.read_text_method = requests_read_method

For each STAC item in the catalog list its date, when it expires, and its access url

In [ ]:
from pystac import Catalog


cat = Catalog.from_file(stac_catalog_url)
print(cat.title)

for item in cat.get_all_items():
    print(item.datetime, item.properties.get('expires'), [asset.href for asset in item.assets.values()])